Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.

##### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Intel® Gaudi® Accelerator Quick Start Guide


This document provides instructions on setting up the Intel Gaudi 2 AI accelerator Instance on the Intel® Developer Cloud or any on-premise Intel Gaudi Node. You will be running models from the Intel Gaudi software Model References and the Hugging Face Optimum Habana library.

Please follow along with the [video](https://developer.habana.ai/intel-developer-cloud/) on our Developer Page to walk through the steps below.  This assumes that you have setup the latest Intel Gaudi PyTorch Docker image.

To set up a multi-node instance with two or more Gaudi nodes, refer to Setting up Multiple Gaudi Nodes in the [Quick Start Guide Documentation](https://docs.habana.ai/en/latest/Intel_DevCloud_Quick_Start/Intel_DevCloud_Quick_Start.html#setting-up-multiple-gaudi-nodeshttps://docs.habana.ai/en/latest/Intel_DevCloud_Quick_Start/Intel_DevCloud_Quick_Start.html#setting-up-multiple-gaudi-nodes).  

The first step is to install the Model-References repository from GitHub and run the "hello-world" model from the examples library.

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Intel_Gaudi_Quickstart
!git clone -b 1.17.1 https://github.com/HabanaAI/Model-References.git

In [ ]:
%cd Model-References/PyTorch/examples/computer_vision/hello_world/

We set the correct paths for the python execution:

In [ ]:
import os
os.environ['PYTHONPATH'] = '$PYTHONPATH:~/Model-References'
os.environ['PYTHON'] = '/usr/bin/python3.10'

We now run the simple example with the MNIST dataset on one Intel Gaudi card:

In [ ]:
!python3 mnist.py --batch-size=64 --epochs=1 --lr=1.0 --gamma=0.7 --hpu --autocast

We can now run the same model on eight Intel Gaudi cards using mpirun:

In [ ]:
!mpirun -n 8 --bind-to core --map-by slot:PE=6 \
      --rank-by core --report-bindings \
      --allow-run-as-root \
      python3 mnist.py \
      --batch-size=64 --epochs=1 \
      --lr=1.0 --gamma=0.7 \
      --hpu --autocast

### Fine-tuning with Hugging Face Optimum Habana Library
The Optimum Habana library is the interface between the Hugging Face Transformers and Diffusers libraries and the Gaudi 2 card. It provides a set of tools enabling easy model loading, training and inference on single and multi-card settings for different downstream tasks. The following example uses the text-classification task to fine-tune a BERT-Large model with the MRPC (Microsoft Research Paraphrase Corpus) dataset and also run Inference.

Follow the below steps to install the stable release from the Optimum Habana examples and library:

1. Clone the Optimum-Habana project and check out the latest stable release.  This repository gives access to the examples that are optimized for Intel Gaudi:

In [1]:
%cd ~
!git clone -b v1.13.2 https://github.com/huggingface/optimum-habana.git

/root
Cloning into 'optimum-habana'...


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


remote: Enumerating objects: 18369, done.
remote: Counting objects: 100% (1954/1954), done.
remote: Compressing objects: 100% (916/916), done.
remote: Total 18369 (delta 1310), reused 1431 (delta 909), pack-reused 16415 (from 1)
Receiving objects: 100% (18369/18369), 11.88 MiB | 9.52 MiB/s, done.
Resolving deltas: 100% (12642/12642), done.
Note: switching to '1266993d741ba97e929965c01307f3c6cce8c107'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



2. Install Optimum-Habana library. This will install the latest stable library:

In [2]:
!pip install --quiet optimum-habana==1.13.2

3. In order to use the DeepSpeed library on Intel Gaudi 2, install the Intel Gaudi DeepSpeed fork:

In [3]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.17.1

  Cloning https://github.com/HabanaAI/DeepSpeed.git (to revision 1.17.1) to /tmp/pip-req-build-_ehmoh8h
  Running command git clone --filter=blob:none --quiet https://github.com/HabanaAI/DeepSpeed.git /tmp/pip-req-build-_ehmoh8h
  Running command git checkout -b 1.17.1 --track origin/1.17.1
  Switched to a new branch '1.17.1'
  Branch '1.17.1' set up to track remote branch '1.17.1' from 'origin'.
  Resolved https://github.com/HabanaAI/DeepSpeed.git to commit e3078cb74d027995725e39806e358a2fceee16be
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.8 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.14.0+hpu.synapse.v1.17.1-py3-none-any.whl size=1445878 sha256=223f43dc4598639fc4bbc6a7fbc81e4adbdfd53c6535a23b3d5891b2a3b29994
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpwpbo8t/wheels/72/74/40/31a4f403d3ece1f3d39f4ae5d87a3cc9752dc00209dc0a9faf
Successfull

The following example is based on the Optimum-Habana Text Classification task example. Change to the text-classification directory and install the additional SW requirements for this specific example:

In [4]:
%cd ~/optimum-habana/examples/text-classification/
!pip install -r requirements.txt

/root/optimum-habana/examples/text-classification
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


### Execute Single-Card Training
This run instruction will fine-tune the BERT-Large Model on one Intel Gaudi card:  

In [5]:
!python run_glue.py \
--model_name_or_path bert-large-uncased-whole-word-masking \
--gaudi_config_name Habana/bert-large-uncased-whole-word-masking  \
--task_name mrpc   \
--do_train   \
--do_eval   \
--per_device_train_batch_size 32 \
--learning_rate 3e-5  \
--num_train_epochs 3   \
--max_seq_length 128   \
--output_dir ./output/mrpc/  \
--use_habana  \
--use_lazy_mode   \
--bf16   \
--use_hpu_graphs_for_inference \
--throughput_warmup_steps 3 \
--report_to none \
--overwrite_output_dir 

[WARNING|utils.py:225] 2024-10-01 02:21:41,553 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but the driver version is v1.18.0, this could lead to undefined behavior!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
gaudi_config.json: 100%|██████████████████████| 90.0/90.0 [00:00<00:00, 380kB/s]
/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  w

### Execute Multi-Card Training
In this example, you will be doing the same fine-tuning task with eight Gaudi 2 cards.   In this case the Optimum-Habana models repository has a helper script called `gaudi_spawn.py` that manages multi card execution.  
Notice the execution time for the fine-tuning compared to the single-card run. 

In [ ]:
!python ../gaudi_spawn.py  --world_size 8 --use_mpi run_glue.py  \
--model_name_or_path bert-large-uncased-whole-word-masking  \
--gaudi_config_name Habana/bert-large-uncased-whole-word-masking  \
--task_name mrpc  \
--do_train  \
--do_eval  \
--per_device_train_batch_size 32  \
--per_device_eval_batch_size 8  \
--learning_rate 3e-5  \
--num_train_epochs 3   \
--max_seq_length 128  \
--output_dir /tmp/mrpc_output/  \
--use_habana   \
--use_lazy_mode   \
--bf16    \
--use_hpu_graphs_for_inference  \
--throughput_warmup_steps 3 \
--report_to none \
--overwrite_output_dir 


[WARNING|utils.py:225] 2024-10-01 02:26:06,593 >> optimum-habana v1.13.2 has been validated for SynapseAI v1.17.0 but the driver version is v1.18.0, this could lead to undefined behavior!
Running with the following model specific env vars: 
MASTER_ADDR=localhost
MASTER_PORT=29500
DistributedRunner run(): command = mpirun -n 8 --bind-to core --map-by socket:PE=10 --rank-by core --report-bindings --allow-run-as-root /usr/bin/python run_glue.py --model_name_or_path bert-large-uncased-whole-word-masking --gaudi_config_name Habana/bert-large-uncased-whole-word-masking --task_name mrpc --do_train --do_eval --per_device_train_batch_size 32 --per_device_eval_batch_size 8 --learning_rate 3e-5 --num_train_epochs 3 --max_seq_length 128 --output_dir /tmp/mrpc_output/ --use_habana --use_lazy_mode --bf16 --use_hpu_graphs_for_inference --throughput_warmup_steps 3 --report_to none --overwrite_output_dir
Authorization required, but no authorization protocol specified
[sc09wynn08-hls2:01902] MCW rank 5 

### Training with DeepSpeed
With the DeepSpeed package already installed, run multi-card training with DeepSpeed. The command below will create and point to a ds_config.json file to set up the parameters of the DeepSpeed run. Once the ds_config.json file is created, you can run the DeepSpeed training command below. 

#### Create DeepSpeed Config file with ZeRO preferences
The ds_config.json file will configure the parameters to run DeepSpeed and will still execute on eight Intel Gaudi 2 Accelerators

In this case, we will run the ZeRO2 optimizer and BF16 mixed precision.

In [ ]:
%%sh
tee ./ds_config.json <<EOF
{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 2,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false
    }
}
EOF

This is the DeepSpeed run command for the bert-fine tuning.   At the completion of the run, compare the runtime and Max Memory usage with the non-DeepSpeed run above, you will see even faster execution and reduced memory consumption.  With larger models these advantages of using DeepSpeed are very important for running Large Language and Generative AI models.

In [ ]:
!python ../gaudi_spawn.py \
--world_size 8 --use_deepspeed run_glue.py \
--model_name_or_path bert-large-uncased-whole-word-masking \
--gaudi_config_name Habana/bert-large-uncased-whole-word-masking \
--task_name mrpc \
--do_train \
--do_eval \
--per_device_train_batch_size 32 \
--per_device_eval_batch_size 8 \
--learning_rate 3e-5 \
--num_train_epochs 3 \
--max_seq_length 128 \
--overwrite_output_dir \
--output_dir /tmp/mrpc_output/ \
--use_habana \
--use_lazy_mode \
--use_hpu_graphs_for_inference \
--throughput_warmup_steps 3 \
--report_to none \
--overwrite_output_dir  \
--deepspeed ds_config.json

### Inference Example Run
This is a separate example using inference only. This will run the same evaluation metrics (accuracy, F1 score) as shown above. This will display how well the model has performed:

In [ ]:
!python run_glue.py --model_name_or_path bert-large-uncased-whole-word-masking \
--gaudi_config_name Habana/bert-large-uncased-whole-word-masking \
--task_name mrpc \
--do_eval \
--max_seq_length 128 \
--output_dir ./output/mrpc/ \
--use_habana \
--use_lazy_mode \
--use_hpu_graphs_for_inference \
--report_to none \
--overwrite_output_dir 

## Next Steps
You now have access to all the Models in Model-References and Optimum-Habana repositories, you can start to look at other models.  Remember that all the models in these repositories are fully documented so they are easy to use.
* To explore more models from the Model References, start [here](https://github.com/HabanaAI/Model-References).  
* To run more examples using Hugging Face go [here](https://github.com/huggingface/optimum-habana?tab=readme-ov-file#validated-models).  
* To migrate other models to Gaudi 2, refer to PyTorch Model Porting in the [documentation](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html)

In [ ]:
# Please be sure to run this exit command to ensure that the resources running on Intel Gaudi are released 
exit()